# Kaggle
## Competição DSA de Machine Learning - Dezembro 2019

Versão 1.0.0: LB = 0.50557
- modelo: XGBoost (com algumas otimizações)
- features categoricas: removido
- dados missing: atribuído o valor medio

Versão 1.0.1: LB = 0.48972 / CV = 0.469777
- modelo: XGBoost executando todas as otimizações
- features engineering: gerado através do Auto_ViML

Versão 1.0.2: LB = 0.55264 / CV = 0.469158
- modelo: XGBoost executando todas as otimizações
- dados missing: removido colunas com mais de 40% de NA e as demais -999
- features categoricas: label encoder
- feature engineering: usando pacote Boruta

Versão 1.0.3: LB = ??? / CV = ???
- modelo: XGBoost executando todas as otimizações
- features engineering: gerado através do Auto_ViML (modificado v1)

Versão 1.0.4: LB = ??? / CV = ???
- modelo: XGBoost executando todas as otimizações
- features engineering: gerado através do Auto_ViML (modificado v2)

## 1. Importando as bibliotecas

In [ ]:
# Importar os principais pacotes
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

import time
import datetime
import gc

# Evitar que aparece os warnings
import warnings
warnings.filterwarnings("ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = True

In [ ]:
# Importa os pacotes de algoritmos
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb

# Importa pacotes do sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, log_loss
from sklearn.preprocessing import scale, MinMaxScaler, StandardScaler


## 2. Carregando os dados de treino e teste

In [ ]:
def read_data():
    
    if TRAIN_OFFLINE:
        print('Carregando arquivo dataset_treino.csv....')
        train = pd.read_csv('../dataset/dataset_treino_modificado.csv')
        print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))
        
        print('Carregando arquivo dataset_teste.csv....')
        test = pd.read_csv('../dataset/dataset_teste_modificado.csv')
        print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))
        
    else:
        print('Carregando arquivo dataset_treino.csv....')
        train = pd.read_csv('/kaggle/input/competicao-dsa-machine-learning-dec-2019/dataset_treino.csv')
        print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))
        
        print('Carregando arquivo dataset_treino.csv....')
        test = pd.read_csv('/kaggle/input/competicao-dsa-machine-learning-dec-2019/dataset_teste.csv')
        print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))
    
    return train, test

In [ ]:
# Leitura dos dados
train, test = read_data()

## 3. Feature Engineering

## 4. Criar e avaliar alguns algoritmos de Machine Learning

In [ ]:
# Separando features preditoras e target
#train_x = train.drop(['ID','target'], axis=1)
train_x = train.drop(['target'], axis=1)
train_y = train['target']

# Padronizando os dados
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)

## 4.1. Algoritmo XGBoost - Extreme Gradient Boosting

Vamos dar uma olhada nas vantagens desse algoritmo:

***Regularização:***
- O XGBoost também é conhecido como uma técnica de "reforço regularizado", ajudando a reduzir o overfitting

***Processamento paralelo:***
- O XGBoost implementa o processamento paralelo e é incrivelmente mais rápido em comparação com o GBM. Mas, espere, sabemos que impulsionar é um processo seqüencial; portanto, como ele pode ser paralelo? Sabemos que cada árvore pode ser construída somente após a anterior, então o que nos impede de fazer uma árvore usando todos os núcleos?

***Alta flexibilidade***
- O XGBoost permite que os usuários definam objetivos de otimização personalizados e critérios de avaliação. Isso adiciona uma nova dimensão ao modelo e não há limite para o que podemos fazer.

***Tratamento de valores ausentes***
- O XGBoost possui uma rotina integrada para lidar com os valores ausentes. É necessário que o usuário forneça um valor diferente de outras observações e passe isso como um parâmetro. O XGBoost tenta coisas diferentes ao encontrar um valor ausente em cada nó e descobre qual caminho seguir para valores ausentes no futuro.

***Poda de árvores:***
- O XGBoost faz divisões até a profundidade máxima especificada e, em seguida, começa a podar a árvore para trás e remove as divisões além das quais não há ganho positivo.

***Validação cruzada incorporada***
- O XGBoost permite que o usuário execute uma validação cruzada a cada iteração do processo de otimização e, portanto, é fácil obter o número ideal exato de iterações de otimização em uma única execução.

- Você pode começar a treinar um modelo XGBoost a partir da última iteração da execução anterior. Isso pode ser uma vantagem significativa em certas aplicações específicas.

In [ ]:
# Criando uma funcao para criação, execução e validação do modelo
def run_model(modelo, X_tr, y_tr, test, useTrainCV=True, cv_folds=5, early_stopping_rounds=10):
    
    # Utilização do Cross-Validation
    if useTrainCV:
        xgb_param = modelo.get_xgb_params()
        xgtrain = xgb.DMatrix(X_tr, label=y_tr)
        
        print ('Start cross validation')
        cvresult = xgb.cv(xgb_param, 
                          xgtrain, 
                          num_boost_round=modelo.get_params()['n_estimators'], 
                          nfold=cv_folds,
                          metrics=['logloss'],
                          stratified=True,
                          seed=42,
                          verbose_eval=True,
                          early_stopping_rounds=early_stopping_rounds)

        modelo.set_params(n_estimators=cvresult.shape[0])
        best_tree = cvresult.shape[0]
        print('Best number of trees = {}'.format(best_tree))
    
    # Fit do modelo
    modelo.fit(X_tr, y_tr, eval_metric='logloss')
        
    # Predição no dataset de treino
    train_pred = modelo.predict(X_tr)
    train_pred_prob = modelo.predict_proba(X_tr)[:,1]
    
    # Exibir o relatorio do modelo
    print("Log Loss (Treino): %f" % log_loss(y_tr, train_pred_prob))
    print("Log Loss (Test): %f" % cvresult['test-logloss-mean'][best_tree-1])
    
    feature_imp = pd.Series(modelo.feature_importances_.astype(float)).sort_values(ascending=False)
    
    plt.figure(figsize=(18,8))
    feature_imp[:25].plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    plt.tight_layout()

#### Passo 01: criando o modelo fixando alguns hyperparametros

In [ ]:
%%time

# Criando o primeiro modelo XGB
modeloXGB = XGBClassifier(learning_rate = 0.1,
                          n_estimators = 1000,
                          max_depth = 5,
                          min_child_weight = 1,
                          gamma = 0,
                          subsample = 0.8,
                          colsample_bytree = 0.8,
                          objective = 'binary:logistic',
                          n_jobs = -1,
                          scale_pos_weight = 1,
                          seed = 42)

run_model(modeloXGB, train_x, train_y)

#### Passo 02: otimização dos parametros: max_depth e min_child_weight

In [ ]:
%%time

# Definindo os parametros que serão testados no GridSearch
param_v1 = {
 'max_depth':range(2,5),
 'min_child_weight':range(1,2)
}

grid_1 = GridSearchCV(estimator = XGBClassifier(learning_rate = 0.1, 
                                                n_estimators = 1000, 
                                                max_depth = 5,
                                                min_child_weight = 1, 
                                                gamma = 0, 
                                                subsample = 0.8, 
                                                colsample_bytree = 0.8,
                                                objective = 'binary:logistic', 
                                                nthread = 4,
                                                scale_pos_weight = 1, 
                                                seed = 42),
                      param_grid = param_v1, 
                      scoring = 'neg_log_loss',
                      n_jobs = -1,
                      iid = False, 
                      cv = 5)

# Realizando o fit e obtendo os melhores parametros do grid
grid_1.fit(train_x, train_y)
grid_1.best_params_, grid_1.best_score_

#### Passo 03: otimização dos parametros: gamma

In [ ]:
%%time

# Definindo os parametros que serão testados no GridSearch
param_v2 = {
 'gamma':[i/10.0 for i in range(0,2)]
}

grid_2 = GridSearchCV(estimator = XGBClassifier(learning_rate = 0.1, 
                                                n_estimators = 1000, 
                                                max_depth = grid_1.best_params_['max_depth'],
                                                min_child_weight = grid_1.best_params_['min_child_weight'], 
                                                gamma = 0, 
                                                subsample = 0.8, 
                                                colsample_bytree = 0.8,
                                                objective = 'binary:logistic', 
                                                nthread = 4, 
                                                scale_pos_weight = 1, 
                                                seed = 42),
                      param_grid = param_v2, 
                      scoring = 'neg_log_loss',
                      n_jobs = -1,
                      iid = False, 
                      cv = 5)

# Realizando o fit e obtendo os melhores parametros do grid
grid_2.fit(train_x, train_y)
grid_2.best_params_, grid_2.best_score_

#### Passo 04: otimização dos parametros: subsample e colsample_bytree

In [ ]:
%%time

# Definindo os parametros que serão testados no GridSearch
param_v3 = {
 'subsample':[i/10.0 for i in range(6,8)],
 'colsample_bytree':[i/10.0 for i in range(6,8)]
}

grid_3 = GridSearchCV(estimator = XGBClassifier(learning_rate = 0.1, 
                                                n_estimators = 1000, 
                                                max_depth = grid_1.best_params_['max_depth'],
                                                min_child_weight = grid_1.best_params_['min_child_weight'], 
                                                gamma = grid_2.best_params_['gamma'], 
                                                subsample = 0.8, 
                                                colsample_bytree = 0.8,
                                                objective = 'binary:logistic', 
                                                nthread = 4, 
                                                scale_pos_weight = 1, 
                                                seed = 42),
                      param_grid = param_v3, 
                      scoring = 'neg_log_loss',
                      n_jobs = -1,
                      iid = False, 
                      cv = 5)

grid_3.fit(train_x, train_y)
grid_3.best_params_, grid_3.best_score_

#### Passo 05: otimização dos parametros: reg_alpha

In [ ]:
%%time

# Definindo os parametros que serão testados no GridSearch
param_v4 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}

grid_4 = GridSearchCV(estimator = XGBClassifier(learning_rate = 0.1, 
                                                n_estimators = 1000, 
                                                max_depth = grid_1.best_params_['max_depth'],
                                                min_child_weight = grid_1.best_params_['min_child_weight'], 
                                                gamma = grid_2.best_params_['gamma'], 
                                                subsample = grid_3.best_params_['subsample'], 
                                                colsample_bytree = grid_3.best_params_['colsample_bytree'],
                                                objective = 'binary:logistic', 
                                                nthread = 4, 
                                                scale_pos_weight = 1, 
                                                seed = 42),
                      param_grid = param_v4, 
                      scoring = 'neg_log_loss',
                      n_jobs = -1,
                      iid = False, 
                      cv = 5)

# Realizando o fit e obtendo os melhores parametros do grid
grid_4.fit(train_x, train_y)
grid_4.best_params_, grid_4.best_score_

## Passo 06: reduzindo Learning Rate

In [ ]:
%%time

# Criando o modelo XGB com todas as otimizações
modeloXGB_v2 = XGBClassifier(learning_rate = 0.01, 
                             n_estimators = 1000, 
                             max_depth = grid_1.best_params_['max_depth'],
                             min_child_weight = grid_1.best_params_['min_child_weight'], 
                             gamma = grid_2.best_params_['gamma'], 
                             subsample = grid_3.best_params_['subsample'], 
                             colsample_bytree = grid_3.best_params_['colsample_bytree'],
                             reg_alpha = grid_4.best_params_['reg_alpha'],
                             objective = 'binary:logistic', 
                             n_jobs = -1,
                             scale_pos_weight = 1, 
                             seed = 42)

run_model(modeloXGB_v2, train_x, train_y)

In [ ]:
# Visualizando o modelo XGBoost Otimizado
print(modeloXGB_v2)

## 4.2. Algoritmo LightGBM

In [ ]:
# Criando uma funcao para criação, execução e validação do modelo LGBM
def run_model_lgb(train_x, train_y, useTrainCV=True, cv_folds=5, early_stopping_rounds=10):
    
    kf = StratifiedKFold(n_splits=cv_folds, shuffle = True, random_state = 42)

    oof_pred = np.zeros((len(train_x), 4))
    y_pred = np.zeros((len(train_y), 4))
    
    for fold, (tr_ind, val_ind) in enumerate(kf.split(train_x, train_y)):
        print('Fold {}'.format(fold + 1))
        
        x_train, x_val = train_x[tr_ind], train_x[val_ind]
        y_train, y_val = train_y[tr_ind], train_y[val_ind]
    
        train_set = lgb.Dataset(x_train, y_train)#, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val)#, categorical_feature=categoricals)

        params = {
            'learning_rate': 0.1,
            'metric': 'logloss',
            'objective': 'binary',
            'num_classes': 1,
            'feature_fraction': 0.75,
            'subsample': 0.75,
            'n_jobs': -1,
            'seed': 42,
            'max_depth': 10
        }

        modelLGB = lgb.train(params, 
                             train_set, 
                             num_boost_round = 1000, 
                             valid_sets=[train_set, val_set], 
                             #early_stopping_rounds = 50, 
                             verbose_eval = 1)
        
        # Predição no dataset de treino
        y_pred = modelLGB.predict(x_val)
        print(y_pred)
        print(x_val)
        print("Log Loss (Treino): %f" % log_loss(x_val, np.argmax(y_pred, axis = 1), eps=1e-15))

        
        oof_pred[val_ind] = modelLGB.predict(x_val)
        y_pred += modelLGB.predict(train_x) / 5
        
        #loss_score = log_loss(train_y, np.argmax(oof_pred, axis = 1), eps=1e-15)
        result = pd.Series(np.argmax(oof_pred, axis = 1))
        #print('Our oof log loss score is: ', loss_score)
        print(result.value_counts(normalize = True))

        # Predição no dataset de teste
        #prev_val = modelLGB.predict(y_val)
        #print("Log Loss (Test): %f" % log_loss(y_val, prev_val, eps=1e-15))
        
        #oof_pred[val_ind] = prev_tr
    
    #log_loss(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None)[source]¶
    #loss_score = log_loss(train_y, np.argmax(oof_pred, axis = 1), eps=1e-15)
    #result = pd.Series(np.argmax(oof_pred, axis = 1))
    
    #print('Our oof log loss score is: ', loss_score)
    #print(result.value_counts(normalize = True))
    
    return y_pred

In [ ]:
%%time

run_model_lgb(train_x, train_y)

## 5. Submissions

In [ ]:
print(modeloXGB_v2.feature_importances_)

In [ ]:
plt.bar(range(len(modeloXGB_v2.feature_importances_)), modeloXGB_v2.feature_importances_)
plt.show()

In [ ]:
feature_imp = pd.Series(modeloXGB_v2.feature_importances_.astype(float)).sort_values(ascending=False)

plt.figure(figsize=(18,8))
feature_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')
plt.tight_layout()

In [ ]:
# Colocando o dataset de teste conforme o modelo treinado
# Neste caso é necessário aplicar a Feature Engineering usada para gerar o modelo
#text_x = test.drop(['ID'], axis=1)

# Removendo todas as variaveis categoricas
#drop_features = []
#for col in text_x.columns:
#    if text_x[col].dtype =='object':
#        drop_features.append(col)
#text_x = text_x.drop(drop_features, axis=1)

# Preenche os dados missing com media
#text_x.fillna(text_x.mean(),inplace=True)
#test = test.iloc[:, :-8]

# Aplicando escala aos dados
text_x = scaler.fit_transform(test)

In [ ]:
submission = pd.read_csv('../dataset/sample_submission.csv')
submission['PredictedProb'] = modeloXGB_v2.predict_proba(text_x)[:,1]
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv('../submission/submission_xgb_v.1.0.1.csv', index=False)

In [ ]:
plt.hist(submission.PredictedProb)
plt.show()